In [2]:
!pip install ctransformers
!pip install ctransformers
!pip install langchain
!pip install faiss-cpu
!pip install faiss-gpu
!pip install sentence-transformers
!pip install transformers
!pip install torch




[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for faiss-gpu



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [45]:
from langchain.llms import CTransformers
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain

# Step 1: Load LLaMA model with optimized settings
def load_llm_local():
    return CTransformers(
        model="C:/Users/chill/llama/llama-2-7b-chat.Q4_K_M.gguf",
        model_type="llama",
        config={
            "max_new_tokens": 100,  # Reduced for speed
            "temperature": 0.7,
            "top_p": 0.95
        }
    )

# Step 2: Simplified prompt
custom_prompt_template = PromptTemplate(
    template="""
<s>[INST] Use the context to answer: {context}
Question: {question} [/INST]
""",
    input_variables=["context", "question"]
)

# Step 3: Load FAISS vector store
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = FAISS.load_local("vectorstore/db_faiss", embeddings=embedding_model, allow_dangerous_deserialization=True)

# Step 4: Wrap LLM in a chain
llm = load_llm_local()
llm_chain = LLMChain(llm=llm, prompt=custom_prompt_template)
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="context")

# Step 5: Build RetrievalQA with k=1
retriever = db.as_retriever(search_kwargs={"k": 1})
qa_chain = RetrievalQA(
    retriever=retriever,
    combine_documents_chain=stuff_chain,
    return_source_documents=True,
    input_key="question"
)

# Step 6: Run the query
user_query = input("Enter your question: ")
response = qa_chain.invoke({"question": user_query})

# Step 7: Print the answer and source documents
print("\nRESULT:\n", response["result"])
print("\nSOURCE DOCUMENTS:")
for doc in response["source_documents"]:
    print("-", doc.metadata.get("source", "Unknown"))


RESULT:
 There are several methods for removing warts, including:
1. Numbing the skin and then scraping off the wart: This is a common method for removing small to medium-sized warts. The area is numbed with a local anesthetic, and then the wart is scrapped off with a scalpel or cryotherapy tool.
2. Electrocautery (electric burning): This method involves using an electric needle to burn

SOURCE DOCUMENTS:
- data\MedicalBook.pdf
